<h2>Analyzing Role Strength and Impact in League of Legends Solo/Duo Queue</h2>

<h3>Background</h3>
<p>League of Legends is a multiplayer online game in which the main objective of a team is to destroy the enemy team's nexus which is located in their base. Each team is made up of five players with unique roles: A top laner, a mid laner, a carry and a support in the botlane, and a jungler. Each role has various functions and provides different value to a team. In the top lane, there is usually a tank character meant to soak damage, in the midlane a damage dealing mage or assassin, and in the jungle role anything in between. In the botlane, the carry is usually a marksman ranged character who plays with a support who provides protection or set up for damage dealers and vision on the map.</p>

<center>
<figure>
  <img src="https://i.imgur.com/niHUZFr.jpg">
  <figcaption>A mini map of Summoner's Rift.</figcaption>
</figure>
</center>

<h3>Motivation</h3>
<p>There is much discussion around imbalance of the powers of different roles in the game. Most notably, the jungle role receives attention for its ability to singlehandedly impact the outcome of a game. But is there any data or statistics that could provide evidence or context for these claims? Is there a significant difference in the ability of a single role to impact the outcome of a game? That is what I set out to discover in this notebook. I also wished to practice pulling data through an API and using Python to manipulate data.</p>

<h3>Method</h3>
<p>In order to assess the impact of a role, I decided that the variables I would use was the gold differential and experience differential of each role in the first 10 minutes, and the target would be whether the team won or not. 

I want to use data from high elo games (Grandmaster-Challenger) because I assume that information from those games will reflect gameplay decisions that are more optimal and standard and decrease variation from that source.</p> 

<h3>Necessary Data</h3>
The data that I require to perform this analysis would be to get the amount of gold, experience, and creeps that each player has at the ten minute mark into the game. 

In [ ]:
#installing packages 
!pip install responses
!pip install requests
!pip install pandas 
!pip install numpy
!pip install time

     |████████████████████████████████| 143kB 6.7MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
     |████████████████████████████████| 133kB 7.2MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.26.3
    Uninstalling urllib3-1.26.3:
      Successfully uninstalled urllib3-1.26.3
ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [ ]:
#importing packages
import responses
import requests
import pandas as pd
import numpy as np
import time

In [ ]:
#set API key
MY_API_KEY = 

In [ ]:
#get the champion keys and corresponding name
champion_info = requests.get("http://ddragon.leagueoflegends.com/cdn/11.2.1/data/en_US/champion.json")
champ_id = champion_info.json()['data']

name = []
id = []
for key in champ_id:
  name.append(champ_id[key]['id'])
  id.append(champ_id[key]['key'])

name_id = {'name': name, 'id': id}
name_ids = pd.DataFrame(data = name_id)

In [ ]:
#sort the champions by their id 
name_ids['id'] = name_ids['id'].astype(int)

name_ids = name_ids.sort_values(by='id')

select = name_ids[name_ids['id'] == 17]
select['name']

122    Teemo
Name: name, dtype: object

In [ ]:
#get current list of challenger players 
chall_players = requests.get(f"https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key={MY_API_KEY}")
player_list = chall_players.json()['entries']
summonerIds = []
for i in range(len(player_list)):
  summonerIds.append(player_list[i]['summonerId'])
                     
len(summonerIds) #there are currently 79 challenger players 

86

In [ ]:
summonerIds

['f-FvU8xZ58Ca5TkkaTkh1zXmTqTspLdXGV0XR_HK2Wo_OzM',
 'hoy1umChFCp1rTJC05AHWyOGJDtLiVVnsw1rjjtyvmOEKgxf',
 'ZAPvmnXcR_uxRmCKK36ffAnmhHpo5BdAGwCYpQIkXkN4u68',
 '0D_zPTxaylKSd-4_QTP-CKIFiu47IqJAqLGGuziJLX0',
 'HMZOo2edoZJzjRTDhBxJpgdYky502ECUKUl5X1BCs-AkHTA',
 '329wFQMZVY93OGLliZT2V4GedGFhRVil6ilSx6U_AnKh-wo',
 'SS7AhtH6MFUy10Ef4qz9tHscZBcdmYfS2TA_RSwCfieH',
 '9LRLMIP3hNBToAWzI7nqjnF-HmyuLyARHFVFYCxxFqf9B6hreY__TeuLHQ',
 'Mst0CSlqNcmHr9r71_s5IYQI2mkBM29eupAXeULg0g4WpoQ',
 'UZI1TvKH624Sr9JdS5R5HDs-r8DY4PKaAppsKoZ6jhVtpWI',
 'w-KFge6OzJvhkR43u1WQuK6q-9Bh5rk8mhrVl6In72O3kvFo3OrJm1Hj5g',
 'LyYegUyrZhdtuXZCPGSVArNiOCnVParXMKutB6Xalu8GOFQ',
 'woV_KAkNLebllBnkt7ixv6xNOZ7Z78I3SykFiTLEz6JajEYB',
 'p0X9sBINzEiAgyStA7B6C0b-HPInMI4EgdHsasHK1W14chw',
 'rgsBNRoRcKs7Ygk9Uz7Vdwl6sEQmGq58CdK8HQp-vDSRAKK8VbQM2G4luw',
 't66u622D_Y4LRy3qX9vnW2mNTIGvvwb7ROi_cksEbatkJLw',
 'QafSpLWkPwq7FkSWcdVXsLmM7GK0-ioTvlXhzI25qi746f4',
 'zrM7gm_S7cYv3CaPEq2aAoKQuUn5EK_73baKhj_Oew5xW_OB',
 'UXdW9W2jhrEdYRJI1nWHqJb9SyngEPsMq

In [ ]:
#get accountIds of the challenger players
accountIds = []
for i in range(len(summonerIds)):
  summoner_info = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summonerIds[i]}?api_key={MY_API_KEY}")
  account_id = summoner_info.json()['accountId']
  accountIds.append(account_id)
  time.sleep(0.5)



In [ ]:
len(accountIds)

86

In [ ]:
#get match history of each player using the accountIds 
match_list = []
for i in range(len(accountIds)):
  player_match_list = requests.get(f"https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/{accountIds[i]}?api_key={MY_API_KEY}")
  matches = player_match_list.json()
  match_list.append(matches)
  time.sleep(0.5)


In [ ]:
len(match_list)

86

In [ ]:
#turn it into one long list of games
player_matches = []

for i in range(len(match_list)):
  try:
    player_matches.append(match_list[i]['matches'])
  except: 
    continue

len(player_matches)



49

In [ ]:
#unroll to get the total amount of games
single_match_data = []

for i in range(len(player_matches)):
  for j in range(len(player_matches[i])):
    single_match_data.append(player_matches[i][j])



In [ ]:
#how many games were there 
len(single_match_data)

4893

In [ ]:
#get id of each match to access the statistics from each game
gameId = []

for i in range(len(single_match_data)):
  gameId.append(single_match_data[i]['gameId'])
  

In [ ]:
#get unique games for no repeats  
gameIds = np.unique(gameId)

len(gameIds)

3381

In [ ]:
#get 100 matches information using the gameIds just for example 
raw_match_data = []

for i in range(100):
  raw_match_info = requests.get(f"https://na1.api.riotgames.com/lol/match/v4/matches/{gameId[i]}?api_key={MY_API_KEY}")
  raw_match_data.append(raw_match_info.json())
  time.sleep(0.5)



In [ ]:
len(raw_match_data)

100

In [ ]:
raw_match_data[0]


In [ ]:
raw_match_data[0]['participantIdentities'][0]['player']['summonerName']

In [ ]:
test_game = raw_match_data[0]

test_game['teams'][0]

In [ ]:
def get_champ_name(id):
  '''replaces champ ids with champ names'''
  name = name_ids.loc[name_ids['id'] == id].values[0]
  return name[0]

In [ ]:
test_name = get_champ_name(15)

test_name

'Sivir'

In [ ]:
#make a match info where we have:the role and champ each player played, their GPM/EPM and if they won the game 

def match_stats(game):
  '''Takes in a single game and retrieves specified information from the game and returns a dataframe of the information'''
  summoner_name = [] #summonerName
  champions = [] #championId
  team = [] #teamId
  gold_per_min = [] #gold per minute
  exp_per_min = [] #experience per minute 
  lane = [] #what lane they played in (guess by riot)
  role = [] #what role they were (guess by riot)
  win = [] #did this player win

  for i in range(10): #loop over each player in the game 
    
    player_info = game['participantIdentities'] #get summoner names 
    summoner_name.append(player_info[i]['player']['summonerName']) 

    player_stats = game['participants'] #get stats about player 
    champions.append(player_stats[i]['championId']) #what champ they played
    champions[i] = get_champ_name(champions[i])

    team.append(player_stats[i]['teamId']) #which team they were on

    player_timeline = player_stats[i]['timeline'] #access the gpm and exp
    gold_per_min.append(player_timeline['goldPerMinDeltas']['0-10'])
    exp_per_min.append(player_timeline['xpPerMinDeltas']['0-10'])
    lane.append(player_timeline['lane'])
    role.append(player_timeline['role'])                   


    teams = game['teams']
    if team[i] == 100:
      win.append(teams[0]['win'])
    else: 
      win.append(teams[1]['win'])
  
  match_info = {'summoner_name': summoner_name, 
                'champion': champions, 
                'team': team,
                'gold_per_min': gold_per_min,
                'exp_per_min': exp_per_min,
                'lane': lane,
                'role': role,
                'win': win
  }

  match_df = pd.DataFrame(data = match_info)

  return match_df


def match_data(match_list):
  '''Takes in a list of games and returns the desired information of each game in a list ?'''
  match_data = []
  
  for i in range(len(match_list)):
    match_data.append(match_stats(match_list[i]))

  return match_data




In [ ]:
match_data = match_data(raw_match_data[0:5])



TypeError: ignored

In [ ]:
print(match_data[0])

     summoner_name     champion  team  ...    lane         role   win
0     xihuan juice        Yuumi   100  ...  MIDDLE  DUO_SUPPORT   Win
1    s0rrymaker123  TwistedFate   100  ...  MIDDLE          DUO   Win
2          TSM Heo        Quinn   100  ...     TOP         SOLO   Win
3        EG deftly        Kaisa   100  ...  MIDDLE          DUO   Win
4   a little st0ry      Taliyah   100  ...  JUNGLE         NONE   Win
5     Romanium 1v9     Renekton   200  ...     TOP         SOLO  Fail
6        agorinass         Yone   200  ...  MIDDLE         SOLO  Fail
7  wqretgdtfdhsdda    Seraphine   200  ...  BOTTOM    DUO_CARRY  Fail
8          Swkeeee          Lux   200  ...  BOTTOM  DUO_SUPPORT  Fail
9    no invade pls      Kindred   200  ...  JUNGLE         NONE  Fail

[10 rows x 8 columns]
